In [ ]:
%pip install -U langchain-community langchain-ollama langchain-qdrant

In [30]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1")

In [ ]:
llm.invoke("What is LangChain?")

In [32]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """\
<|start_header_id|>system<|end_header_id|>
You are a helpful assistant. You answer user questions based on provided context. If you can't answer the question with the provided context, say you don't know.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
User Query:
{query}

Context:
{context}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [33]:
rag_chain = rag_prompt | llm

In [ ]:
rag_chain.invoke({"query" : "Who old is Carl?", "context" : "Carl is a sweet dude, he's 40."})

In [35]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [36]:
from langchain_community.document_loaders import TextLoader

document_loader = TextLoader("./data/paul_graham_essays.txt")
documents = document_loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
split_documents = text_splitter.split_documents(documents)
len(split_documents)

In [39]:
from langchain_qdrant import QdrantVectorStore

url = "localhost:6333"
qdrant = QdrantVectorStore.from_documents(
    split_documents,
    embeddings,
    url=url,
    prefer_grpc=True,
    collection_name="Paul_Graham_Documents",
)

In [ ]:
from langchain_qdrant import QdrantVectorStore

url = "localhost:6333"
qdrant_vectorstore = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="Paul_Graham_Documents",
    prefer_grpc=True,
    url=url
    )